In [1]:
# Importing libraries and establishing connection to SQL database

import numpy as np
import pandas as pd
import sqlite3 as sql
import difflib
data_path = '../zippedData'
conn = sql.connect(data_path + '/im.db')

In [2]:
# Declaration of base dataframes

# Studios and gross revenue - 3387 rows
# https://www.boxofficemojo.com/
gross_df = pd.read_csv(data_path + '/bom.movie_gross.csv')
# The movie database - https://www.themoviedb.org/?language=en-US
# Reviews - 26517 rows, release date, popularity score, votes?
reviews_df = pd.read_csv(data_path + '/tmdb.movies.csv')
# Budget and gross revenues - 5782 rows
# https://www.the-numbers.com/
budget_df = pd.read_csv(data_path + '/tn.movie_budgets.csv')
# Decided these 2 were unnecessary
# Movie ratings (R, etc.), genre, director, writer, theater/dvd date, length, box office amount, studio - 1560 rows
movie_info_df = pd.read_csv(data_path + '/rt.movie_info.tsv', sep='\t')
# # Rotten tomatoes - reviews, crtiic, review rating, critic quality - 54,432 rows
# rt_reviews_df = pd.read_csv(data_path + '/rt.reviews.tsv', sep='\t', encoding='latin-1')
# Ratings and genres by director from imdb database
director_and_ratings_df = pd.read_sql(
"""SELECT DISTINCT b.*,
            r.averagerating AS avg_rating,
            r.numvotes AS num_votes,
            p.primary_name AS director,
            p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN directors d
ON b.movie_id = d.movie_id
LEFT JOIN persons p
ON d.person_id = p.person_id
""", conn)
# Ratings and genres by writers from imdb database
writers_df = pd.read_sql("""
SELECT DISTINCT b.*, r.averagerating AS avg_rating, r.numvotes AS num_votes, p.primary_name AS 'writer', p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN writers w
ON b.movie_id = w.movie_id
LEFT JOIN persons p
ON w.person_id = p.person_id

""", conn)

In [3]:
movie_info_df

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [4]:
reviews_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [5]:
budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [6]:
budget_df.loc[budget_df.movie.str.lower() == 'avatar']

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


In [7]:
gross_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [8]:
# pd.merge(gross_df, director_and_ratings_df, )

In [9]:
director_and_ratings_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,director,primary_profession
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,Mani Kaul,"director,writer,actor"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,Orson Welles,"actor,director,writer"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,Hrishikesh Mukherjee,"director,editor,writer"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,Raoul Ruiz,"director,writer,producer"
...,...,...,...,...,...,...,...,...,...,...
86772,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6,Giancarlo Soldi,"director,writer,producer"
86773,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,Ahmet Faik Akinci,"director,writer"
86774,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
86775,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"


In [10]:
writers_df.loc[writers_df.primary_title.str.lower() == 'avatar']

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,writer,primary_profession
17859,tt1775309,Avatar,Abatâ,2011,93.0,Horror,6.1,43,Yûsuke Yamada,writer
17860,tt1775309,Avatar,Abatâ,2011,93.0,Horror,6.1,43,Teruo Noguchi,"director,writer,editor"


In [11]:
writers_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,avg_rating,num_votes,writer,primary_profession
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Abrar Alvi,"writer,actor,director"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Gulzar,"music_department,writer,soundtrack"
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Mahasweta Devi,writer
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Anjana Rawail,"writer,costume_designer"
4,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,None,None
...,...,...,...,...,...,...,...,...,...,...
122379,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
122380,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Victoria Sloan Jordan,"writer,producer"
122381,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Coral Cruz,"writer,miscellaneous"
122382,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Pep Puig,writer


In [12]:
reviews_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [13]:
# From https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee
genre_dictionary = [{"id":28,"name":"Action"},
                              {"id":12,"name":"Adventure"},
                              {"id":16,"name":"Animation"},
                              {"id":35,"name":"Comedy"},
                              {"id":80,"name":"Crime"},
                              {"id":99,"name":"Documentary"},
                              {"id":18,"name":"Drama"},
                              {"id":10751,"name":"Family"},
                              {"id":14,"name":"Fantasy"},
                              {"id":36,"name":"History"},
                              {"id":27,"name":"Horror"},
                              {"id":10402,"name":"Music"},
                              {"id":9648,"name":"Mystery"},
                              {"id":10749,"name":"Romance"},
                              {"id":878,"name":"Science Fiction"},
                              {"id":10770,"name":"TV Movie"},
                              {"id":53,"name":"Thriller"},
                              {"id":10752,"name":"War"},
                              {"id":37,"name":"Western"}]

In [14]:
reviews_df.genre_ids.str.strip('\[\]').str.replace(" ", "").str.split(',')

0            [12, 14, 10751]
1        [14, 12, 16, 10751]
2              [12, 28, 878]
3            [16, 35, 10751]
4              [28, 878, 12]
                ...         
26512               [27, 18]
26513               [18, 53]
26514           [14, 28, 12]
26515        [10751, 12, 28]
26516               [53, 27]
Name: genre_ids, Length: 26517, dtype: object

In [15]:
# reviews_df.genre_ids = reviews_df.genre_ids.str.strip('\[\]').str.replace(" ", "").str.strip().str.split(',')
# reviews_df = reviews_df.explode('genre_ids')
# reviews_df.genre_ids = reviews_df.genre_ids.replace(r'^\s*$', np.nan, regex=True)
# reviews_df.genre_ids.astype('float')
# def find_genre(key):
#     if key is np.nan:
#         return 'None'
#     for dictionary in genre_dictionary:
#         if dictionary['id'] == int(key):
#             return dictionary['name']
#     return 'None'

# reviews_df.genre_ids = reviews_df.genre_ids.apply(lambda x: find_genre(x))


In [16]:
len(reviews_df.title.unique())

24688

In [17]:
reviews_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [18]:
budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [19]:
gross_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [20]:
# Reformatting data in budget_df

# Turned all the monetary strings into ints
budget_df[budget_df.columns[3:]] = budget_df[budget_df.columns[3:]].replace('[\$\.]|,', '', regex=True).astype('int')
# Added column of world wide gross - production budget to get the net profit
budget_df['profit'] = budget_df['worldwide_gross'] - budget_df['production_budget']

In [21]:
# Merging budget dataframe and gross dataframe by domestic gross
# Merged on movie from budget_df and title from gross_df
merged_budget_and_gross_df = pd.merge(budget_df, gross_df, how = "outer", left_on = 'movie', right_on = 'title')
# Dropping id and year columns, since year is represented in release_date (may need to go back, since some release_dates are missing)
merged_budget_and_gross_df.drop(['id', 'year'], axis=1, inplace=True)
# Merging gross_y and gross_x columns, preferring gross_x which seem to have higher precision
merged_budget_and_gross_df.domestic_gross_x = merged_budget_and_gross_df.domestic_gross_x.fillna(merged_budget_and_gross_df.domestic_gross_y)
# Merging movie and title, preferring movie
merged_budget_and_gross_df.movie = merged_budget_and_gross_df.movie.fillna(merged_budget_and_gross_df.title)
# Dropping redundant gross and title columns
merged_budget_and_gross_df.drop(['domestic_gross_y', 'title'], axis=1, inplace=True)
# Change dates to datetime
merged_budget_and_gross_df['release_date'] = pd.to_datetime(merged_budget_and_gross_df['release_date']).dt.strftime('%Y-%m-%d')
# New column for year
merged_budget_and_gross_df['year'] = merged_budget_and_gross_df['release_date'].str.split('-').str[0]
merged_budget_and_gross_df

,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,profit,studio,foreign_gross,year
0,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,NaN,NaN,2009
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,BV,804600000,2011
2,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08,NaN,NaN,2019
3,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09,BV,946400000,2015
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08,NaN,NaN,2017
...,...,...,...,...,...,...,...,...,...
7926,NaN,The Quake,NaN,6200.0,NaN,NaN,Magn.,NaN,NaN
7927,NaN,Edward II (2018 re-release),NaN,4800.0,NaN,NaN,FM,NaN,NaN
7928,NaN,El Pacto,NaN,2500.0,NaN,NaN,Sony,NaN,NaN
7929,NaN,The Swan,NaN,2400.0,NaN,NaN,Synergetic,NaN,NaN


In [22]:
person_df = pd.merge(writers_df, director_and_ratings_df, how='inner', on='movie_id')
person_df

,movie_id,primary_title_x,original_title_x,start_year_x,runtime_minutes_x,genres_x,avg_rating_x,num_votes_x,writer,primary_profession_x,primary_title_y,original_title_y,start_year_y,runtime_minutes_y,genres_y,avg_rating_y,num_votes_y,director,primary_profession_y
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Abrar Alvi,"writer,actor,director",Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Gulzar,"music_department,writer,soundtrack",Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Mahasweta Devi,writer,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Anjana Rawail,"writer,costume_designer",Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
4,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,None,None,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,Mani Kaul,"director,writer,actor"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182134,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor",Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
182135,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Victoria Sloan Jordan,"writer,producer",Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
182136,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Coral Cruz,"writer,miscellaneous",La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"
182137,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Pep Puig,writer,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"


In [23]:
reviews_df['release_date'] = pd.to_datetime(reviews_df['release_date']).dt.strftime('%Y-%m-%d')

In [24]:
merged_budget_and_gross_df.movie = merged_budget_and_gross_df.movie.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.lower()
person_df.primary_title_x = person_df.primary_title_x.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.lower()
reviews_df.title = reviews_df.title.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.lower()

In [25]:
# director_and_ratings_df.primary_title = director_and_ratings_df.primary_title.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.lower()

In [26]:
merged_budget_and_gross_df = merged_budget_and_gross_df.loc[merged_budget_and_gross_df.production_budget.notna()]

In [27]:
type(merged_budget_and_gross_df.release_date[0])

str

In [28]:
type(reviews_df.release_date[0])

str

In [29]:
person_df.start_year_x = person_df.start_year_x.astype(str)

In [30]:
merge_reviews = pd.merge(merged_budget_and_gross_df, reviews_df, how='left', left_on=['movie', 'release_date'] , right_on=['title', 'release_date'])
merge_reviews_directors = pd.merge(merge_reviews, person_df, how='left', left_on=['movie', 'year'], right_on=['primary_title_x','start_year_x'])

In [31]:
merge_reviews

,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,profit,studio,foreign_gross,year,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,title,vote_average,vote_count
0,2009-12-18,avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,NaN,NaN,2009,6.0,"[28, 12, 14, 878]",19995.0,en,Avatar,26.526,avatar,7.4,18676.0
1,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,BV,804600000,2011,2470.0,"[12, 28, 14]",1865.0,en,Pirates of the Caribbean: On Stranger Tides,30.579,pirates of the caribbean on stranger tides,6.4,8571.0
2,2019-06-07,dark phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08,NaN,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-05-01,avengers age of ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09,BV,946400000,2015,14169.0,"[28, 12, 878]",99861.0,en,Avengers: Age of Ultron,44.383,avengers age of ultron,7.3,13457.0
4,2017-12-15,star wars ep viii the last jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08,NaN,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5911,2018-12-31,red 11,7000.0,0.0,0.000000e+00,-7.000000e+03,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5912,1999-04-02,following,6000.0,48482.0,2.404950e+05,2.344950e+05,NaN,NaN,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5913,2005-07-13,return to the land of wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03,NaN,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5914,2015-09-29,a plague so pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
merge_reviews_directors

,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,profit,studio,foreign_gross,year,Unnamed: 0,...,primary_profession_x,primary_title_y,original_title_y,start_year_y,runtime_minutes_y,genres_y,avg_rating_y,num_votes_y,director,primary_profession_y
0,2009-12-18,avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,NaN,NaN,2009,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,BV,804600000,2011,2470.0,...,"writer,director,producer",Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011.0,136.0,"Action,Adventure,Fantasy",6.6,447624.0,Rob Marshall,"director,miscellaneous,producer"
2,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,BV,804600000,2011,2470.0,...,"writer,producer,miscellaneous",Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011.0,136.0,"Action,Adventure,Fantasy",6.6,447624.0,Rob Marshall,"director,miscellaneous,producer"
3,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,BV,804600000,2011,2470.0,...,"writer,actor",Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011.0,136.0,"Action,Adventure,Fantasy",6.6,447624.0,Rob Marshall,"director,miscellaneous,producer"
4,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,BV,804600000,2011,2470.0,...,"writer,director,producer",Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011.0,136.0,"Action,Adventure,Fantasy",6.6,447624.0,Rob Marshall,"director,miscellaneous,producer"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9444,2018-12-31,red 11,7000.0,0.0,0.000000e+00,-7.000000e+03,NaN,NaN,2018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9445,1999-04-02,following,6000.0,48482.0,2.404950e+05,2.344950e+05,NaN,NaN,1999,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9446,2005-07-13,return to the land of wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03,NaN,NaN,2005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9447,2015-09-29,a plague so pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03,NaN,NaN,2015,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
merge_reviews_directors.columns

Index(['release_date', 'movie', 'production_budget', 'domestic_gross_x',
       'worldwide_gross', 'profit', 'studio', 'foreign_gross', 'year',
       'Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'title', 'vote_average', 'vote_count', 'movie_id',
       'primary_title_x', 'original_title_x', 'start_year_x',
       'runtime_minutes_x', 'genres_x', 'avg_rating_x', 'num_votes_x',
       'writer', 'primary_profession_x', 'primary_title_y', 'original_title_y',
       'start_year_y', 'runtime_minutes_y', 'genres_y', 'avg_rating_y',
       'num_votes_y', 'director', 'primary_profession_y'],
      dtype='object')

In [34]:
columns_to_keep = ['release_date','production_budget','movie', 'domestic_gross_x', 'worldwide_gross', 'profit', 'genre_ids', 'runtime_minutes_x', 'genres_x', 'writer', 'director']
columns_to_drop = [x for x in list(merge_reviews_directors.columns) if x not in columns_to_keep]

In [35]:
merge_reviews_directors.drop(columns_to_drop, axis=1, inplace=True)

In [36]:
merge_reviews_directors

,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,profit,genre_ids,runtime_minutes_x,genres_x,writer,director
0,2009-12-18,avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,"[28, 12, 14, 878]",NaN,NaN,NaN,NaN
1,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,"[12, 28, 14]",136.0,"Action,Adventure,Fantasy",Stuart Beattie,Rob Marshall
2,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,"[12, 28, 14]",136.0,"Action,Adventure,Fantasy",Ted Elliott,Rob Marshall
3,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,"[12, 28, 14]",136.0,"Action,Adventure,Fantasy",Tim Powers,Rob Marshall
4,2011-05-20,pirates of the caribbean on stranger tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,"[12, 28, 14]",136.0,"Action,Adventure,Fantasy",Terry Rossio,Rob Marshall
...,...,...,...,...,...,...,...,...,...,...,...
9444,2018-12-31,red 11,7000.0,0.0,0.000000e+00,-7.000000e+03,NaN,NaN,NaN,NaN,NaN
9445,1999-04-02,following,6000.0,48482.0,2.404950e+05,2.344950e+05,NaN,NaN,NaN,NaN,NaN
9446,2005-07-13,return to the land of wonders,5000.0,1338.0,1.338000e+03,-3.662000e+03,NaN,NaN,NaN,NaN,NaN
9447,2015-09-29,a plague so pleasant,1400.0,0.0,0.000000e+00,-1.400000e+03,NaN,NaN,NaN,NaN,NaN


In [37]:
column_names = ['release_date', 'title', 'prod_budget', 'dom_gross', 'world_gross', 'profit', 'genre_ids', 'runtime_min', 'genres', 'writer', 'director']
merge_reviews_directors.columns = column_names
merge_reviews_directors = merge_reviews_directors.reindex(columns=['title', 'profit', 'prod_budget', 'dom_gross', 'world_gross', 'genre_ids', 'genres','director','writer','runtime_min','release_date'])

In [38]:
merge_reviews_directors.drop_duplicates(inplace=True)

In [41]:
merge_reviews_directors

,title,profit,prod_budget,dom_gross,world_gross,genre_ids,genres,director,writer,runtime_min,release_date
0,avatar,2.351345e+09,425000000.0,760507625.0,2.776345e+09,"[28, 12, 14, 878]",NaN,NaN,NaN,NaN,2009-12-18
1,pirates of the caribbean on stranger tides,6.350639e+08,410600000.0,241063875.0,1.045664e+09,"[12, 28, 14]","Action,Adventure,Fantasy",Rob Marshall,Stuart Beattie,136.0,2011-05-20
2,pirates of the caribbean on stranger tides,6.350639e+08,410600000.0,241063875.0,1.045664e+09,"[12, 28, 14]","Action,Adventure,Fantasy",Rob Marshall,Ted Elliott,136.0,2011-05-20
3,pirates of the caribbean on stranger tides,6.350639e+08,410600000.0,241063875.0,1.045664e+09,"[12, 28, 14]","Action,Adventure,Fantasy",Rob Marshall,Tim Powers,136.0,2011-05-20
4,pirates of the caribbean on stranger tides,6.350639e+08,410600000.0,241063875.0,1.045664e+09,"[12, 28, 14]","Action,Adventure,Fantasy",Rob Marshall,Terry Rossio,136.0,2011-05-20
...,...,...,...,...,...,...,...,...,...,...,...
9444,red 11,-7.000000e+03,7000.0,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2018-12-31
9445,following,2.344950e+05,6000.0,48482.0,2.404950e+05,NaN,NaN,NaN,NaN,NaN,1999-04-02
9446,return to the land of wonders,-3.662000e+03,5000.0,1338.0,1.338000e+03,NaN,NaN,NaN,NaN,NaN,2005-07-13
9447,a plague so pleasant,-1.400000e+03,1400.0,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2015-09-29


In [42]:
merge_reviews_directors.to_csv('../zippedData/merged_data.csv')

In [39]:
merge_reviews_directors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9085 entries, 0 to 9448
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         9085 non-null   object 
 1   profit        9085 non-null   float64
 2   prod_budget   9085 non-null   float64
 3   dom_gross     9085 non-null   float64
 4   world_gross   9085 non-null   float64
 5   genre_ids     3965 non-null   object 
 6   genres        4818 non-null   object 
 7   director      4820 non-null   object 
 8   writer        4798 non-null   object 
 9   runtime_min   4806 non-null   float64
 10  release_date  9085 non-null   object 
dtypes: float64(5), object(6)
memory usage: 851.7+ KB


In [40]:
# merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.str.strip('\[\]').str.replace(" ", "").str.strip().str.split(',')
# merge_reviews_directors = merge_reviews_directors.explode('genre_ids')
# reviews_df.genre_ids = reviews_df.genre_ids.replace(r'^\s*$', np.nan, regex=True)
# reviews_df.genre_ids.astype('float')
# def find_genre(key):
#     if key is np.nan:
#         return 'None'
#     for dictionary in genre_dictionary:
#         if dictionary['id'] == int(key):
#             return dictionary['name']
#     return 'None'

# reviews_df.genre_ids = reviews_df.genre_ids.apply(lambda x: find_genre(x))